In [1]:
from gapfinder.convert import convert_tif_to_png
import os

# convert the files from the tif format to the png format

The images should be placed into folders like the examples

In [2]:
input_folder = "./images/tif_images"
output_folder = "./images/raw_images"

In [ ]:
# Extract metadata from folder structure
# images\tif_images\Wild Type - 500uE 1 hour\2022-1-10\Block 1\13 WT-500uE 29k.tif
# genotype_condition would be: ['Wild Type', '500uE 1 hour']
# genotype would be: 'Wild Type'
# condition would be: '500uE 1 hour'
# date would be: '2022-1-10'
# block would be: '1'
# layer_number would be: '13'    
# k_value would be: '29'
    
convert_tif_to_png(input_folder, output_folder, csv_filename='./metadata/image_index.csv')

# select the grana locations of interest

Select the grana centroids by running the next block. It will create a popup, and allow you to cycle through the images and select the grana centroids. The data will be saved in the `centroids.csv` file in the input image folder.

Available commands:
* n (or tab, or t): Move to the next image, saving the ROI list for this image.
* r (or e): Reset the current image and remove the ROIs.
* q: Quit the program and save the ROI data processed for all images so far.

In [ ]:
from gapfinder.grana_selection import process_images

input_folder = './images/raw_images'
output_filename = 'centroids.csv'
verbose = True
append = False

# delete the existing centroids? Or append to them?
if (append == False):
    if os.path.exists(output_filename):
        os.remove(output_filename)

process_images(input_folder, output_filename, verbose)

# Use the centroids to extract a region of interest around that point

The extract_strips_from_images takes the centroids and extracts a region of interest around that point. The region of interest is a square of size `size` centered at the centroid. The extracted images are saved in the `strips` folder in the input image folder. If the region of interest would go outside the image, the image is skipped.

In [ ]:
from gapfinder.strip_extraction import extract_strips_from_images

raw_images_folder = './images/raw_images'
centroids_csv = f"{raw_images_folder}/centroids.csv"
strip_images_folder = './images/strip_images'

extract_strips_from_images(centroids_csv, raw_images_folder, strip_images_folder, size=200)

# Rotate the roi images to align the grana

This requires a human's input. The images are displayed, and the user can rotate the image to align the grana. The rotated images are saved in the `rotated` folder in the input image folder, and will be used later by the scripts.

Available commands:
* s: Save the current rotation and move to the next image.
* r: Reset the current image and remove the line.
* q: Quit the program and save the rotation angles processed so far.
* h: Display this help message again.

In [ ]:
from gapfinder.strip_rotation import rotate_strips

# Call the function with the desired arguments
input_folder = './images/strip_images'
output_folder = './images/rotated_images'
rotation_file = 'rotation_angles.txt'

rotate_strips(input_folder, output_folder, rotation_file)

# After rotation, select the roi boundaries

Iterate through the rotated images, selecting the region boundaries. This data will be saved in the output file specified. 

* Press 's', 'Tab', or '9' to save the ROI coordinates and move to the next image.
* Press 'n' to skip to the next image.
* Press 'r' to reset the current image, erasing the roi rectangle.
* Press 'q' to quit the program.
* Press 'h' to display these hints.

In [ ]:
from gapfinder.roi_boundaries import select_rois

# Call the function with the desired arguments
image_folder = './images/rotated_images'
rotation_file = './images/rotated_images/rotation_angles.txt'
metadata_file = './images/strip_images/strip_metadata.csv'
output_file = './metadata/roi_metadata.csv'
roi_context_dir = './images/roi_images_context'

select_rois(image_folder, rotation_file, metadata_file, output_file, roi_context_dir)

# After selecting the roi boundaries, extract the roi from the original png images

The extract_rois_from_images script extracts the roi from the original images. The roi is saved in the `rois` folder in the input image folder.

In [2]:
from gapfinder.roi_extraction import extract_rois

roi_metadata_csv = './metadata/roi_metadata.csv'
output_roi_metadata = './metadata/roi_metadata_2.csv'
rotated_images_folder = './images/rotated_images'
output_folder = './images/roi_images'

extract_rois(roi_metadata_csv, output_roi_metadata, rotated_images_folder, output_folder)

Processing completed. New ROI metadata saved to './metadata/roi_metadata_2.csv'
